In [3]:
getwd() # working directory 
ls()    # currently nothing in my R workspace

[1] "/Users/dsp/dongmei/climate-space/notebooks"

character(0)

In [4]:
source('../R/damian/getPseudoDaily.R')  # load .R source file

# now the functions/objects in the file are available
ls()                                    

[1] "get.a.j"                "get.a.nought"           "get.b.j"               
[4] "get.daily.from.monthly" "interpolate.daily"